## Allgemeine Vorarbeiten
Zu Beginn werden Pakete in der virtuellen Umgebung installiert, die benötigt werden. Beispielsweise "pandas" über den Befehl "%pip install pandas". Anschließend werden in einer separaten Zelle die benötigten Pakete importiert.

In [ ]:
# Install basic packages if not already installed
%pip install pandas
%pip install numpy
%pip install matplotlib

In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

Zunächst werden alle allgemeinen Konstanten und Funktionen definiert.

In [3]:
# Setting up some constants used also in the following functions
# fine structure constant
alpha = 1/137
# Mass of the electron in MeV
m_e = 0.511
# Radius of the electron in 1/fm
r_e = 1.44/m_e
# Radius of the electron in cm
r_e = r_e * 1e-13
# Avogadro's number
N_A = 6.022*10**23

# Coulomb correction function for electron/positron
def f_coul(Z):
    a = Z/137
    f = a**2*((1+a**2)**(-1)+0.20206-0.0369*a**2+0.0083*a**4-0.002*a**6)
    return f

# Separate helper function for the common factor of the Bethe-Bloch formula
def bethe_bloch_common_factor():
    # Calculate the common factor
    K = 2 * np.pi * N_A * r_e**2 * m_e
    return K

# Bethe-Bloch formula for the energy loss
def bethe_bloch(Z_det, A_det, rho_det, Z_projectile, beta_projectile, W_max, I_potential, density_correction, coulomb_correction):
    # Calculate beta_gamma
    beta_gamma = beta_projectile / np.sqrt(1 - beta_projectile**2)
    # Calculate common factor
    K = 2 * np.pi * N_A * r_e**2 * m_e
    dE = K * Z_det / A_det * rho_det * Z_projectile**2 / beta_projectile**2 * (np.log(2 * m_e * beta_gamma**2 * W_max / I_potential**2) - 2 * beta_projectile**2 - density_correction - 2 * coulomb_correction/Z_det)
    return dE

# number of atoms in 1 cm^3
def rho_to_N(rho, A):
    # Calculate number of atoms per cm^3
    N = rho * N_A / A
    return N
# Sollte 2,7*10^19 sein

# Function for the radiative correction
def phi_rad(Z, r_e, alpha, E0, m_e, shield_ind):
    # Calculate common factor
    cf = 4 * Z**2 * r_e**2 * alpha
    # Calculate the radiative correction
    if shield_ind == 0:
        phi = cf * (np.log(2 * E0 / m_e) - 1/3 - f_coul(Z))
    elif shield_ind == 1:
        phi = cf * (np.log(183*Z**(-1/3)) + 1/18 - f_coul(Z))
    return phi

# Function for the energy loss
def dE_dx(E0, Z, A, rho, r_e, alpha, m_e, shield_ind):
    # Calculate the energy loss
    dE = rho_to_N(rho, A) * E0 * phi_rad(Z, r_e, alpha, E0, m_e, shield_ind)
    return dE

# Approximation of the critical energy for electrons
def E_crit_approx(Z):
    # Calculate the critical energy using an approximation
    E_crit = 800/(Z+1.2)
    return E_crit

# Simplified radiation length function
def L_rad(Z, A):
    # Calculate the radiation length
    X0 = 716.4 * A / (Z*(Z+1) * np.log(287/np.sqrt(Z)))
    return X0

# Bethe-Heitler distribution function
def bh_distribution(E, E0, Z, A, material_length):
    # Calculate the length in terms of the radiation length
    t = material_length / L_rad(Z, A)
    # Calculate the relative energy
    x = E / E0
    # Calculate the distribution
    dN = (-np.log(x))**(t/np.log(2)-1)/math.gamma(t/np.log(2))
    return dN 

# Function to calculate the maximum energy loss (or energy transfer)
def E_max(beta_gamma, gamma, m_e, M):
    # Calculate the maximum energy transfer under the assumption of natural units
    # where c = 1 and masses are in MeV/c^2
    E_max = 2 * m_e * beta_gamma**2 / (1 + 2 * gamma * (m_e/M) + (m_e/M)**2)
    return E_max

## Blatt 2
Für die Aufgabenteile
1. vollständige Abschirmung, d.h. `shield_ind = 1`
2. keine Abschirmung, d.h. `shield_ind = 0`

In [5]:
# Initial energy in MeV
E0 = 1000
Z = 21 # Atomic number of Scandium
# Atomic mass of Scandium
A = 43
rho = 1.95*10**-3  # g/cm^3
# Avogadro's number
N_A = 6.022*10**23  # atoms/mol


# Exercise part a) complete shielding
print('Aufgabenteil a) ' "%10.3e"% dE_dx(E0, Z, A, rho, r_e, alpha, m_e, 1) + " MeV/cm")
# Exercise part b) no shielding
print('Aufgabenteil b) ' "%10.3e"% dE_dx(E0, Z, A, rho, r_e, alpha, m_e, 0) + " MeV/cm")

Aufgabenteil a)  1.179e-01 MeV/cm
Aufgabenteil b)  2.209e-01 MeV/cm


Aufgabenteil c): Die Voraussetzung der Formel muss geprüft werden, d.h. $E_0 >> 137 m_c \cdot c^2 \cdot Z^{1/3}$.

In [6]:
print('Aufgabenteil d) ' "%1.1f"% E_crit_approx(Z) + ' MeV')

Aufgabenteil d) 36.0 MeV


Die kritisch Teilchenenergie ist die Energie, bei der der Energieverlust durch Strahlung gleich dem Energieverlust durch Streuung entspricht. Elektronen bzw. Myonen mit E = 1 GeV liegen somit immer oberhalb der kritischen Energie.
Man müsste hier den WQ über die Abschätzung aus dem Hinweis nehmen und das Verhältnis von $E_c(e^-)/E_c(\mu)$ betrachtet. Daraus folgert man dann $E_c(\mu)\approx 1,5$ TeV.

In [ ]:
Z = 14 # Atomic number of Silicon
A = 28
rho = 2.33*10**-3  # g/cm^3

# number of slices
n_slices = 6
# width of the slices
slice_width = 300 # in µm
# number of slices in cm
n_slices_cm = n_slices * slice_width * 1e-4
# Initial energy in MeV
E0 = 1000

# X0 = Lrad/rho = 9,45 cm
# t = DeltaX/X0 = 1,9%
# --> in der e) sollte dann 3,9% rauskommen, bei der f) rund 8% aus t = 2*1,9%

# Calculate the probability of a 50% energy loss after 6 slices
print('Aufgabenteil e) ' "%f"% bh_distribution(E0/2, E0, Z, A, n_slices_cm) + ' %')
# Calculate the probability of a 50% energy loss after 12 slices
print('Aufgabenteil f) ' "%f"% bh_distribution(E0/2, E0, Z, A, 2*n_slices_cm) + ' %')

Aufgabenteil e) 0.017062 %
Aufgabenteil f) 0.034198 %


Aufgabe 2 betrachtet Energieauflösungen, um die $K_\alpha$-Linie $E_\gamma\approx 5.9$ keV mit einer Energieauflösung von $\Delta E_{\text{rel.}}<20\%$ zu messen.
a) Die deponierte Energie $E_\gamma$ wird auf das Ionisationspotential bezogen, um die Anzahl der Elektronen/Löcher zu bestimmen, d.h. $N=\frac{E_\gamma}{I_{pot}}$ -> Vola bestimmen ($\Delta E$) über Poisson-Statistik ($\frac{1}{\sqrt{N}}$) --> bspw. Si-HLD ergibt 146 eV --> 2,4%
b) Gasdetektor ist hier wegen Zusammenspiel mit Energieauflösung vs. freie Wegstrecke zu bevorzugen.

In [8]:
5900/3.6

1638.888888888889

## Blatt 3
### Aufgabe 1
#### a)
Der *Vavilov*-Parameter $\kappa$ ist ein Maß für das Verhältnis (nicht das Verhältnis selbst!) von mittlerem Energieverlust zum maximalen Energieverlust, d.h.
\begin{equation}
\kappa = \frac{\xi}{T_{\max}}
\end{equation}
mit $\xi = 2\pi N_a r_e^2 m_e c^2 \rho \frac{Z}{A}(\frac{z}{\beta})^2 \Delta x$.
Für kleine Werte von $\kappa$ geht die *Vavilov*-Verteilung in die *Landau*-Verteilung über.

#### b)
Der *Vavilov*-Parameter soll für minimal ionisierende Pionen in einem Si-Detektor der Dicke $\Delta x=900\,\mu m$ mit einer Geschwindigkeit von $\beta \approx 0,97, \beta\gamma = 4$ bestimmt werden, um zu entscheiden, ob die *Vavilov*-Verteilung mit der *Landau*-Verteilung genähert werden darf.

In [ ]:
# Parameter
beta = 0.97
beta_gamma = 4
gamma = 1 / math.sqrt(1 - beta**2)
detector_thickness = 900 # in µm
# Convert to cm
detector_thickness_cm = detector_thickness * 1e-4
# Mass of the particle (here pion) in MeV/c^2
M_pion = 139.57
# Charge of the particle (here pion), which is equal to the atomic number of the particle
q_pion = 1
# Information about the detector
# Atomic number of Silicon
Z_det = 14
# Atomic mass of Silicon
A_det = 28
# Density of Silicon
rho_det = 2.33*10**-3  # g/cm^3

# Calculate xi
xi = bethe_bloch_common_factor() * Z_det / A_det * rho_det * q_pion**2 / beta**2 * detector_thickness_cm
# maximum energy loss
E_max_pion = E_max(beta_gamma, gamma, m_e, M_pion)
print('Maximale Energie ' "%1.1f"% E_max_pion + ' MeV')
# Vavilov factor
kappa = xi/E_max_pion
print('Vavilov Faktor: ', kappa, '. Der Faktor nahe 0 legt nahe, dass die Energieverlustverteilung eine Landauverteilung ist.')

# Der Wert sollte so um die 1,046*10^(-3) liegen...

Maximale Energie 15.9 MeV
Vavilov Faktor:  1.0778853207711233e-06 . Der Faktor nahe 0 legt nahe, dass die Energieverlustverteilung eine Landauverteilung ist.


### Aufgabe 2
#### a)
Die mittlere freie Weglänge $\lambda=\frac{1}{\mu}=\frac{1}{n\sigma}$ mit $\sigma$ dem Streuquerschnitt und $n=\rho\frac{N_a}{A}$ der Teilchendichte kann für ein bekanntes Material (hier Argon, Z=18) bestimmt werden, wenn man die Energy der Photonen kennt. Für die $^{55}Fe$-$K_{\alpha}$-Linie mit einer Energie $E_{\gamma}=5{,}9\,$ keV kann aus Messreihen (z.B. unter https://physics.nist.gov/PhysRefData/XrayMassCoef/ElemTab/z18.html) der normierte Absorptionskoeffizient $\frac{\mu_{en}}{\rho}$ bestimmt werden.
5,9\,keV entsprechen rund $6\cdot 10^{-3}\,$ MeV, woraus sich ein normierter Absorptionskoeffizient von $\frac{\mu_{en}}{\rho}=2{,}449\cdot 10^2\,cm^2/g$ ergibt.

In [5]:
# Energy in keV
E = 5.9
# Energy in MeV
E_MeV = E * 1e-3
# Atomic mass of Argon
A_Ar = 40
# effective cross section in cm^2
sigma = 5*10**2 * A_Ar/N_A
# Calculate the effective cross section in barns
sigma_barns = sigma / 1e-24
print('Effektive Querschnittsfläche: ' "%1.1e"% sigma + ' cm^2')
# density of Argon
rho_Ar = 1.784*10**-3  # g/cm^3
# absorption coefficient
mu = rho_Ar * N_A/A_Ar * sigma
print('Absorptionskoeffizient: ' "%1.6f"% mu + ' cm^-1')
# Calculate the mean free path
lambda_mfp = 1/mu
print('Mittlere freie Weglänge: ' "%1.1f"% lambda_mfp + ' cm')

Effektive Querschnittsfläche: 3.3e-20 cm^2
Absorptionskoeffizient: 0.892000 cm^-1
Mittlere freie Weglänge: 1.1 cm


#### b)
Ein Gas mit einer geringeren freien Weglänge als 2,3 cm ist gesucht. Dichtere Gase würden hier helfen. Xenon sollte genommen werden.

In [6]:
# Atomic mass of Xenon
A_Xe = 131
# effective cross section in cm^2
sigma = 5.085*10**2 * A_Xe/N_A
# Calculate the effective cross section in barns
print('Effektive Querschnittsfläche: ' "%1.1e"% sigma + ' cm^2')
# density of Xenon
rho_Xe = 2.9*10**-3  # g/cm^3
# absorption coefficient
mu = rho_Xe * N_A/A_Xe * sigma
print('Absorptionskoeffizient: ' "%1.6f"% mu + ' cm^-1')
# Calculate the mean free path
lambda_mfp = 1/mu
print('Mittlere freie Weglänge: ' "%1.1f"% lambda_mfp + ' cm')

Effektive Querschnittsfläche: 1.1e-19 cm^2
Absorptionskoeffizient: 1.474650 cm^-1
Mittlere freie Weglänge: 0.7 cm
